# Import Libraries

In [37]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

from dotenv import load_dotenv
import chromadb
import json
import os

from pprint import pprint

from few_shots import few_shots
load_dotenv()

True

_____

# Setup LLM

In [38]:
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=1,
    groq_api_key=os.getenv("GROQ_API_KEY")  
)

_________________

# Job Description in JSON

In [39]:
loader = WebBaseLoader("https://careers.nike.com/senior-ai-ml-engineer/job/R-66196")
page_data = loader.load().pop().page_content

page_data = "\n".join(
    line.strip() for line in page_data.splitlines() if line.strip()
)

print(page_data)

Senior AI/ML Engineer
Skip to main content
Open Virtual Assistant
Home
Career Areas
Total Rewards
Life@Nike
Purpose
Language
Select a Language
Deutsch
English
Español (España)
Español (América Latina)
Français
Italiano
Nederlands
Polski
Tiếng Việt
Türkçe
简体中文
繁體中文
עִברִית
한국어
日本語
Careers
Close Menu
Careers
Chat
Home
Career Areas
Total Rewards
Life@Nike
Purpose
Jordan Careers
Converse Careers
Language
Menu
Return to Previous Menu
Select a Language
Deutsch
English
Español (España)
Español (América Latina)
Français
Italiano
Nederlands
Polski
Tiếng Việt
Türkçe
简体中文
繁體中文
עִברִית
한국어
日本語
Back to Search
Senior AI/ML Engineer
Categories ID
Categories URL
Position Type
Full Time
Date Posted
Primary Quest ID
Second Quest ID
Job Classification
Career area
Software Engineering
Location
Remote
Job ID
R-66196
Apply Now
Share Job
Share Job Posting
Facebook
Opens In A New Tab
LinkedIn
Link Opens In New Window
Email
Close-Medium (Default Size)-icon
Close Menu
Become a Part of the NIKE, Inc. Team
NIKE, 

In [40]:
prompt_extract = PromptTemplate(
    input_variables=["page_data"],
    template="""
### Scraped Text from Web url
{page_data}

# Instruction
The scraped text is from the career's page of a website.
Your job is to extract the job postings and return them in JSON format containing the 
following keys: `role`, `experience`, `skills in detail`, and `description`. 
Only return the valid JSON.

### VALID JSON (NO PREAMBLE):
"""
)

In [41]:
json_parser = JsonOutputParser() # eventhough the llm will output in json format it will still be str so we need to pass jasonparser.

chain_extract = prompt_extract | llm | json_parser 

job_description = chain_extract.invoke(input={'page_data':page_data})

pprint(job_description)
print(f"\ntype: {type(job_description)})")

{'description': 'Influence and develop robust machine learning and generative '
                'AI solutions that have a direct impact on the business. '
                'Design and implement scalable applications that leverage '
                'prediction models and optimization programs to deliver data '
                'driven decisions that result in immense business impact.',
 'experience': '5 to 7 years of professional experience in software '
               'engineering',
 'role': 'Senior AI/ML Engineer',
 'skills in detail': {'agile development': 'Test-driven development',
                      'cloud platforms': 'Amazon Web Services',
                      'containerization': 'Docker',
                      'data processing technologies': 'Spark, FastAPI',
                      'database technologies': 'Postgres, Redis',
                      'machine learning frameworks': 'Scikit-learn, PyTorch, '
                                                     'Tensorflow',
          

___________________

# Resume in Json

In [42]:
with open("my_resume.txt", "r", encoding="utf-8") as f:
    resume_text = f.read()

resume_to_json_prompt = PromptTemplate(
    input_variables=["resume_text"],
    template="""
You are an assistant that converts resumes into structured JSON.
Each section or subheading in the resume should become a key.
The content under each heading should become the value.
Preserve all important details.

Resume Text:
{resume_text}

Return only valid JSON without any extra text.
"""
)

In [44]:
resume_chain = resume_to_json_prompt | llm | json_parser
resume = resume_chain.invoke({"resume_text": resume_text})

resume

{'Contact Information': {'Name': 'Rohit Karthick',
  'Location': 'Chennai, Tamil Nadu, India',
  'Phone': '+91-9566887888',
  'Email': 'rkbeo5@gmail.com',
  'LinkedIn': 'https://www.linkedin.com/in/rohit-karthick/',
  'Github': 'https://github.com/7rohxt'},
 'Technical Skills': {'Languages': 'Python, SQL',
  'ML/DL': 'TensorFlow, Keras, Scikit-learn, OpenCV, Transformers',
  'Web/API': 'FastAPI, Flask, Streamlit',
  'DevOps': 'Docker, AWS EC2, Firebase, Git',
  'LLM Tools': 'LangChain, LoRA fine-tuning'},
 'Professional Experience': [{'Company': 'NeubAItics Pvt. Tech. Ltd.',
   'Location': 'Chennai, India',
   'Roles': [{'Role': 'Machine Learning Intern',
     'Duration': 'Jan 2025 to Jun 2025',
     'Responsibilities': ['Assisted in building a real-time person tracking system using DeepSORT and InsightFace across zones and multiple CCTV cameras.',
      'Built and dockerized a real-time face attendance system with spoof detection using OpenCV and FastAPI; integrated Firebase for secur

_________________

# Generate Cold Email (Zero-shot)

In [45]:
email_prompt = PromptTemplate(
    input_variables=["job_description", "resume"],
    template="""
You are the candidate described in the resume.

Job Description:
{job_description}

Candidate Resume:
{resume}

Instruction:
Write a concise, professional cold email to the hiring manager that clearly explains
why the candidate is a strong fit for this specific role. 
"""
)

email_chain = email_prompt | llm
email_response = email_chain.invoke({"job_description": job_description, "resume": resume})

print(email_response.content)

Subject: Application for Senior AI/ML Engineer Role

Dear Hiring Manager,

I am excited to apply for the Senior AI/ML Engineer position at your esteemed organization. With a strong background in machine learning and a passion for developing robust AI solutions, I am confident that I would be an excellent fit for this role.

As a seasoned machine learning engineer with experience in building and deploying scalable AI applications, I possess a deep understanding of the skills required to influence and develop machine learning and generative AI solutions that drive business impact. My expertise in Python, TensorFlow, Scikit-learn, and Docker aligns with the job requirements, and my experience with FastAPI, AWS, and containerization will enable me to design and implement scalable applications that leverage prediction models and optimization programs.

My professional experience at NeubAItics Pvt. Tech. Ltd. has provided me with hands-on experience in building real-time person tracking syst

__________________

# Vector Database

In [53]:
chroma_client = chromadb.PersistentClient(path="chroma_db") 

embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

collection = chroma_client.get_or_create_collection(name="few_shots")

ids = []
documents = []
metadatas = []

for i, example in enumerate(few_shots):
    job_text = json.dumps(example["job_description"], indent=2)
    role = example["job_description"].get("role", "unknown") 
    
    ids.append(f"example_{i}")
    documents.append(job_text)
    metadatas.append({
        "example_id": f"example_{i}",
        "role": role,
        "resume": json.dumps(example["resume"]),
        "email": example["email"]
    })

collection.add(
    ids=ids,
    documents=documents,
    metadatas=metadatas
)

### Retrive one example with most similar Job description

In [54]:
query = "AI/ML Engineer with AWS and Python"

results = collection.query(
    query_texts=[query],
    n_results=2
)

print("Top matches:\n", results)

Top matches:
 {'ids': [['example_1', 'example_0']], 'embeddings': None, 'documents': [['{\n  "role": "AI/ML Engineer",\n  "experience": "2-5 years of professional experience in software engineering",\n  "skills in detail": [\n    "Python",\n    "Algorithms and data structures",\n    "AWS experience",\n    "Database technology (e.g. Postgres, Redis)",\n    "Data processing technology (e.g. Sagemaker or DataBricks)",\n    "MLOps",\n    "Cloud architecture and technologies, especially Amazon Web Services: ECR, SageMaker, Lambda, API Gateway",\n    "Frameworks such as Scikit-learn, PyTorch, Tensorflow, Spark, FastAPI",\n    "Pipeline orchestration tools such as AirFlow or Databricks Workflows",\n    "Spark, Kubernetes, Docker, Jenkins, Databricks, or Terraform"\n  ],\n  "description": "Influence and develop robust machine learning and generative AI solutions that have a direct impact on the business, designing and implementing scalable applications that leverage prediction models and optim

### Unpack retrived example

In [55]:
retrieved_job_description = results["documents"][0][0]
retrieved_meta = results["metadatas"][0][0]

retrieved_resume = retrieved_meta["resume"]
retrieved_email  = retrieved_meta["email"]

_____

# Build Few Shot Prompt

_______________

In [48]:
retrieved_examples = []
for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    retrieved_examples.append({
        "retrieved_job_description": doc,
        "retrieved_resume": meta["resume"],
        "retrieved_email": meta["email"]
    })

example_prompt = PromptTemplate(
    input_variables=["retrieved_job_description", "retrieved_resume", "retrieved_email"],
    template="""
Example Job Description:
{retrieved_job_description}

Example Candidate Resume:
{retrieved_resume}

Example Email:
{retrieved_email}
---
"""
)

few_shot_prompt = FewShotPromptTemplate(
    examples=retrieved_examples,
    example_prompt=example_prompt,
    suffix="""
Now, write a new email for the following job:

Job Description:
{job_description}

Candidate Resume:
{resume}

Instructions:
- Highlight how the candidate's skills, frameworks, and technologies match the job requirements exactly.
- Connect relevant projects or experience to each key requirement from the job description.
- If the candidate does not meet the required years of experience, emphasize related projects and achievements.
- Keep it short, persuasive, and directly aligned with the job posting.
- Include contact details below the name (email, phone, LinkedIn, GitHub if available).
- Avoid generic statements; make it specific to the job description.
""",
    input_variables=["job_description", "resume"]
)

final_prompt = few_shot_prompt.format(
    job_description=job_description,
    resume=resume
)

email_chain = few_shot_prompt | llm
email_response = email_chain.invoke({
    "job_description": job_description,
    "resume": resume
})
print(email_response.content)

KeyError: '\n  "role"'

# Generate Cold Email with Few Shot Learning

In [56]:
from langchain.prompts import PromptTemplate

few_shot_prompt = PromptTemplate(
    input_variables=[
        "retrieved_job_description",
        "retrieved_resume",
        "retrieved_email",
        "job_description",
        "resume"
    ],
    template="""
You are the candidate described in the resume.

### Reference Example
Job Description:
{retrieved_job_description}

Candidate Resume:
{retrieved_resume}

Cold Email:
{retrieved_email}

---

### Task
Now, write a new cold email for the following role:

Job Description:
{job_description}

Candidate Resume:
{resume}

### Guidelines
- Emphasize how the candidate’s skills, frameworks, and technologies align with the job requirements.
- Connect relevant projects/experience to each key requirement in the job description.
- If required years of experience are missing, highlight equivalent projects and achievements instead.
- Keep the email concise, persuasive, and tailored to this job (avoid generic phrasing).
- End with candidate contact details (email, phone, LinkedIn, GitHub if available).
"""
)


In [ ]:
email_chain = few_shot_prompt | llm
email_response = email_chain.invoke({
    "retrieved_job_description": retrieved_job_description,
    "retrieved_resume": retrieved_resume,
    "retrieved_email": retrieved_email,
    "job_description": job_description,
    "resume": resume
})

print(email_response.content)

Subject: Application for Senior AI/ML Engineer Role – Leveraging Python, AWS, and ML Expertise

Dear Hiring Manager,

I am excited to apply for the Senior AI/ML Engineer position, where I can utilize my expertise in Python, machine learning, and cloud computing to drive business growth. With a strong foundation in programming languages, cloud platforms, and machine learning frameworks, I am confident in my ability to develop robust AI solutions.

My experience with Python, Scikit-learn, PyTorch, and Tensorflow aligns well with your requirements. I have successfully deployed projects using these frameworks, including a LoRA fine-tuned LLM for domain-specific text understanding and a Text-to-SQL agent that converts natural language queries into SQL. Additionally, my work with Docker and AWS EC2 demonstrates my proficiency in containerization and cloud deployment.

As a skilled engineer, I have developed scalable applications that leverage prediction models and optimization programs. My p